In [1]:
#_*_coding: utf-8_*_
import pandas as pd
import numpy as np
from selenium import webdriver #从selenium中调入webdriver

In [2]:
from selenium.webdriver.common.keys import Keys

In [3]:
import time

In [4]:
browser = webdriver.Chrome('C:/Users/rk/Desktop/chromedriver.exe') #锁定浏览器

In [5]:
browser.get("https://gaokao.chsi.com.cn/sch/search.do?searchType=1&start=1960") #访问页面

In [6]:
result=[]

In [7]:
counter=99#页码初始化
count=0#当前页url

In [25]:
while True:
    url_nowelements=browser.find_elements_by_xpath("//tbody/tr//a[@target='_blank']")#获取整个页面的高校url元素信息
    url_now=[]#存储当前页面高校url
    for url_nowelement in url_nowelements:
        url_nowpage=[]#存储当前高校校名及url的临时列表
        url_nowpage.append(url_nowelement.text)#存储校名
        url_nowpage.append(url_nowelement.get_attribute('href'))#获取学校url并存储到url_nowpage中
        url_now.append(url_nowpage)#将存储校名，url的临时列表存储到url_now中
    for url in url_now[count:]:#访问当前页面的学校，设置count是为了防止访问到某个学校后出问题，重新执行后从此学校开始
        time.sleep(1)
        try:#防止打开网页出现问题
            browser.get(url[1])#访问相应学校
        except:
            time.sleep(1)
            coun+=1
            if coun>=5:
                break
        flag=0#获取网页数据格式不一致，基本上存在四种形式（定位网址元素有四种不同的路径）
        try:
            webaddr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/a').text
            url.append(webaddr)
            addr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[2]/span').text
            url.append(addr)
            url.append('one')#后面将数据写入TXT只能是字符串，数字不行，
        except:
            flag=1#第一中情况没有找到网址，flag设置为1，接着查找第二中情况
        if flag==1:
            try:
                webaddr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p[1]/a').text
                url.append(webaddr)
                addr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[2]/span').text
                url.append(addr)
                url.append('two')
            except:
                flag=2
                
        if flag==2:
            try:
                webaddr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p/span').text
                url.append(webaddr)
                addr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[2]/span').text
                url.append(addr)
                url.append('three')
            except:
                flag=3
        if flag==3:
            try:
                webaddr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p').text
                url.append(webaddr)
                addr=browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[2]/span').text
                url.append(addr)
                url.append('four')
            except:
                flag=4
        if flag==4:#如果四种情况都没有找到相应网址，就相应处理一下，为了最后生成dataframe不出问题
            url.append('NA')
            url.append('NA')
            url.append('NA')
        print(url)#输出每个学校的查询结果
        with open(r'C:\Users\rk\Desktop\university0329.txt','a+') as f:#及时保存查询结果
            str='、'.join(url)#列表生成字符串
            f.write(str+'\n')#write只能保存字符串，不能保存列表
        result.append(url)#将每所学校的查询结果保存到最终的列表里
        browser.back()#查询一所学校结束后就后退网页，防止出现仅显示本科院校的情况
        time.sleep(0.5)
        count+=1#每访问一所学校就加1，防止访问学校出现崩溃情况而重新执行
    count=0#当前页面访问完成后，count置为0
    counter+=1#访问页面所有学校后，当前页码增加1
    if counter>137:#当前页码大于最后页码后跳出循环
        break
    time.sleep(3)
    #browser.get("https://gaokao.chsi.com.cn/sch/search.do?searchType=1&yxmc=&zymc=&sySsdm=&ssdm=&yxls=&yxlx=&xlcc=bk") 
    browser.find_element_by_xpath('//*[@id="p_num"]').clear()#清空页码输入框
    browser.find_element_by_xpath('//*[@id="p_num"]').send_keys(counter)#输入页码
    browser.find_element_by_xpath('/html/body/div[2]/div[3]/div/div/div/form/ul//input[2]').click()#访问页码

['酒泉职业技术学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1086.dhtml', 'http://www.jqzy.com/', '甘肃省酒泉市肃州区解放路66号', 'one']
['兰州外语职业学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1313.dhtml', 'http://www.lzwyedu.com/', '甘肃省兰州市和平开发区薇乐大道137号', 'two']
['兰州职业技术学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1314.dhtml', 'http://www.lvu.edu.cn/', '兰州市安宁区刘沙公路37号', 'one']
['甘肃警察职业学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1315.dhtml', 'http://www.gs-police.com/', '兰州市城关区左家湾169号', 'one']
['甘肃林业职业技术学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1316.dhtml', 'http://www.gsfc.edu.cn/', '甘肃省天水市麦积区马跑泉路58号', 'one']
['甘肃工业职业技术学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1317.dhtml', '甘肃工业职业技术学院（http://www.gipc.edu.cn/）', '甘肃省天水市羲皇大道中段', 'two']
['西北师范大学知行学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1782.dhtml', 'http://zxxy.nwnu.edu.cn/default_zsb.asp', '西北师范大学知行学院招生办公室 地址：甘肃省兰州市安宁区长新路20号 邮编：730070', 'four']
['兰州商学院陇桥学院', 'https://gaokao.chsi.co

['新疆大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-590.dhtml', 'http://www.xju.edu.cn/', '新疆乌鲁木齐市胜利路666号', 'one']
['塔里木大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-591.dhtml', 'http://www.taru.edu.cn/', '新疆维吾尔自治区阿拉尔市塔里木大道', 'one']
['新疆农业大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-592.dhtml', 'http://www.xjau.edu.cn/', '乌鲁木齐市农大东路311号', 'one']
['石河子大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-593.dhtml', 'http://www.shzu.edu.cn/', '新疆石河子市北四路 石河子大学招生办公室', 'two']
['新疆医科大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-594.dhtml', 'http://www2.xjmu.edu.cn/', '新疆乌鲁木齐市新医路393号', 'one']
['新疆师范大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-595.dhtml', 'http://www.xjnu.edu.cn', '(新医路校区)乌鲁木齐市沙依巴克区新医路102号 (文光校区)乌鲁木齐市新市区喀什东路16号 (温泉校区)乌鲁木齐市温泉东路', 'one']
['喀什大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-596.dhtml', 'http://www.ksu.edu.cn/', '喀什市学院路29号', 'two']
['伊犁师范学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-597.dhtml', 'http://w

In [21]:
print(count)

12


In [24]:
print(counter)

131


In [23]:
count+=1
print(count)

13


In [ ]:
['甘肃医学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-915.dhtml', 'http://www.plmc.edu.cn/', '甘肃省平凉市崆峒区柳湖路东路38号', 'one']
['陇南师范高等专科学校', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-916.dhtml', 'http://www.lntc.edu.cn/', '地址：甘肃省成县河东区陇南路34号 邮政编码：742500', 'two']
['兰州工业学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-917.dhtml', 'http://www.lzit.edu.cn/', '兰州市七里河区龚家坪东路1号 邮编：730050', 'two']
['定西师范高等专科学校', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-918.dhtml', 'http://www.dxatc.cn/', '甘肃省定西市定临路4号', 'one']

In [ ]:
广西培贤国际职业学院
重庆传媒
四川大学锦江学院
甘肃建筑职业技术学院

In [22]:
print(counter)

115


In [20]:
counter=110

In [23]:
counter+=1

In [ ]:
运城学院

In [ ]:
内蒙古


In [ ]:
河南护理职业学院
106:广西培贤。。。桂林生命与健康职业技术学院

In [ ]:
109：重庆警察。。。
115：四川大学锦江学院。。。。


In [374]:
browser.find_element_by_xpath('/html/body/div[2]/div[2]/div/h1/a').text

'北京邮电大学\ue835'

In [32]:
 browser.find_element_by_xpath('/html/body/div[2]/div[3]/div/div/div/form/ul//input[2]').click()

In [20]:
a=['s','b']

In [23]:
'、'.join(a)

's、b'

In [28]:
browser.back()

In [7]:
url_nowelements=browser.find_elements_by_xpath("//tbody/tr//a[@target='_blank']")#

In [8]:
url_nowelements[0].text

'北京大学'

In [9]:
url_now=[]
for url_nowelement in url_nowelements:
    url_nowpage=[]
    url_nowpage.append(url_nowelement.text)
    url_nowpage.append(url_nowelement.get_attribute('href'))
    url_now.append(url_nowpage)

In [10]:
url_now

[['北京大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-1.dhtml'],
 ['中国人民大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-2.dhtml'],
 ['清华大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-3.dhtml'],
 ['北京交通大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-4.dhtml'],
 ['北京工业大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-5.dhtml'],
 ['北京航空航天大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-6.dhtml'],
 ['北京理工大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-7.dhtml'],
 ['北京科技大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-8.dhtml'],
 ['北方工业大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-9.dhtml'],
 ['北京化工大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-10.dhtml'],
 ['北京工商大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-11.dhtml'],
 ['北京服装学院', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-12.dhtml'],
 ['北京邮电大学', 'https://gaokao.chsi.com.cn/sch/schoolInfo--schId-13.dhtml'],
 ['北京印刷学院', 'https://gaokao.chsi.com.cn/sch/schoo

In [11]:
i=13

In [12]:
browser.get(url_now[i][1]) #

In [13]:
browser.back()

In [78]:
browser.find_element_by_xpath('//*[@id="p_num"]').clear()

In [79]:
browser.find_element_by_xpath('//*[@id="p_num"]').send_keys(130)

In [81]:
browser.find_element_by_xpath('/html/body/div[2]/div[3]/div/div/div/form/ul/li[10]/input[2]').click()

In [391]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[2]/span').text

'北京市大兴区兴华大街（二段）1号'

In [332]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/a').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/a"}
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=72.0.3626.7 (efcef9a3ecda02b2132af215116a03852d08b9cb),platform=Windows NT 10.0.17134 x86_64)


In [333]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p[1]/a').text

'www.cfau.edu.cn'

In [277]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p/span').text

'http://www.cau.edu.cn '

In [278]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p').text

'http://www.cau.edu.cn  招办网站：http://jwzs.cau.edu.cn/'

In [239]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/a').text

'www.ncut.edu.cn'

In [232]:
browser.find_element_by_xpath('/html/body/div[2]/div[4]/div[1]/div[2]/div[1]/span[1]/p/span').text

'zhaosheng.ustb.edu.cn'